In [1]:
import pandas as pd

In [2]:
# URLs for the data
patron_path = r'C:\Users\Ben\IdeaProjects\hrp-machine-learning\data\wh_data\Patron_Checkouts.xlsx'

patron_df = pd.read_excel(patron_path)

In [3]:
print(len(patron_df))
patron_df.head()

90878


,Patron Barcode,Title,Author's Name,Holdings Barcode,Date of Action,Unnamed: 5,Unnamed: 6
0,10010,Of mice and men,"Steinbeck, John,",120181,01/19/2021 03:57:26PM,NaN,NaN
1,10010,ILL - The truth at daybreak,"Torockio, Christopher",90000718,03/11/2021 12:19:29PM,NaN,NaN
2,10010,ILL - The truth at daybreak,"Torockio, Christopher",90000718,04/08/2021 04:24:14PM,NaN,NaN
3,10041,The unforgotten : a novel,"Powell, Laura",123448,05/01/2021 11:43:20AM,NaN,NaN
4,10041,"Star Wars, the clone wars.","Hibbert, Clare,",131727,05/01/2021 11:43:22AM,NaN,NaN


In [4]:
# Remove entries that flow into extra columns
patron_df_trim = patron_df[patron_df['Unnamed: 5'].isnull()]
patron_df_trim = patron_df[patron_df['Unnamed: 6'].isnull()]

# Remove extra columns and rename remaining columns
patron_df_trim = patron_df.drop(columns=['Holdings Barcode', 'Date of Action', 'Unnamed: 5', 'Unnamed: 6'])
patron_df_trim = patron_df_trim.rename(columns={'Patron Barcode': 'patron_id', 'Title': 'title', 'Author\'s Name': 'author'})

In [5]:
# Drop entries with null values
patron_df_trim = patron_df_trim.dropna()

In [6]:
# Split mutiple authors on ; and author first and last on ,
patron_df_trim['author'] = patron_df_trim['author'].str.split(';', expand=True)[0]
patron_df_split = patron_df_trim

patron_df_split['author_last'] = patron_df_split['author'].str.split(',', expand=True)[0]
patron_df_split['author_first'] = patron_df_split['author'].str.split(',', expand=True)[1]

patron_df_split = patron_df_split.drop(columns = ['author'])

In [7]:
patron_df_split.head()

,patron_id,title,author_last,author_first
0,10010,Of mice and men,Steinbeck,John
1,10010,ILL - The truth at daybreak,Torockio,Christopher
2,10010,ILL - The truth at daybreak,Torockio,Christopher
3,10041,The unforgotten : a novel,Powell,Laura
4,10041,"Star Wars, the clone wars.",Hibbert,Clare


In [8]:
patron_df_split = patron_df_split.dropna()
len(patron_df_split)

76211

In [9]:
# Drop ILL entries
patron_df_split = patron_df_split[patron_df_split['title'].str.contains('ILL') == False]

In [10]:
# Convert all strings to lowercase
patron_df_split['title'] = patron_df_split['title'].str.lower()
patron_df_split['author_last'] = patron_df_split['author_last'].str.lower()
patron_df_split['author_first'] = patron_df_split['author_first'].str.lower()

In [11]:
patron_df_split.head()

,patron_id,title,author_last,author_first
0,10010,of mice and men,steinbeck,john
3,10041,the unforgotten : a novel,powell,laura
4,10041,"star wars, the clone wars.",hibbert,clare
5,10041,"star wars, the clone wars.",beecroft,simon.
7,10041,wow! : the pop-up book of sports,foster,bruce.


In [12]:
# Remove all special characters
patron_df_split['title'] = patron_df_split['title'].str.replace('[^a-z]', ' ', regex = True)
patron_df_split['author_last'] = patron_df_split['author_last'].str.replace('[^a-z]', ' ', regex = True)
patron_df_split['author_first'] = patron_df_split['author_first'].str.replace('[^a-z]', ' ', regex = True)

# Remove repeated white space
patron_df_split['title'] = patron_df_split['title'].replace(r'\s+', ' ', regex=True)
patron_df_split['author_last'] = patron_df_split['author_last'].replace(r'\s+', ' ', regex=True)
patron_df_split['author_first'] = patron_df_split['author_first'].replace(r'\s+', ' ', regex=True)

In [13]:
# Assign each patron a new id
patron_list = list(set(patron_df_split['patron_id'].tolist()))
ids = {patron: i for i, patron in enumerate(patron_list)}

patron_df_id = patron_df_split.replace({'patron_id': ids})

In [14]:
# Create and inventory df
inv_df = patron_df_id[['title', 'author_last', 'author_first']]
inv_df = inv_df.drop_duplicates()

In [15]:
# Reset the index of inv_df to be 0 based
inv_df = inv_df.reset_index().drop(columns = ['index'])

In [16]:
# Swap out book info for index stored in inv_df
patron_df_id = pd.merge(patron_df_id, inv_df.reset_index())
patron_df = patron_df_id.drop(columns = ['title', 'author_last', 'author_first'])
patron_df = patron_df.rename(columns = {'index': 'item_id'})

In [17]:
patron_df.to_csv(r'C:\Users\Ben\IdeaProjects\hrp-machine-learning\data\clean_data\patron_data.csv')
inv_df.to_csv(r'C:\Users\Ben\IdeaProjects\hrp-machine-learning\data\clean_data\inv_data.csv')